In [1]:
# imports
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

10000 - arts and entertainment

12009 - education
12004 - community center
12080 - library
19046 - metro station
15014 - hospital


13032 - cafes, coffee, tea houses
13029 - brewery
13003 - bar
13065 - restaurant

16000 - landmarks and outdoors
18000 - sports and rec

17057 - retail food/drink

17029 - convenience store
17035 - drug store



In [93]:
bike_df = pd.read_csv('bike_df')
bike_df.head()

,Station_name,Latitude,Longitude,Bikes,Coord_String
0,10th & Cambie,49.262487,-123.114397,35,49.262487%2C-123.114397
1,Yaletown-Roundhouse Station,49.274566,-123.121817,16,49.274566%2C-123.121817
2,Dunsmuir & Beatty,49.279764,-123.110154,26,49.279764%2C-123.110154
3,12th & Yukon (City Hall),49.260599,-123.113504,16,49.260599%2C-123.113504
4,8th & Ash,49.264215,-123.117772,16,49.264215%2C-123.117772


In [94]:
testbike_df = bike_df.head(10)
testbike_df

,Station_name,Latitude,Longitude,Bikes,Coord_String
0,10th & Cambie,49.262487,-123.114397,35,49.262487%2C-123.114397
1,Yaletown-Roundhouse Station,49.274566,-123.121817,16,49.274566%2C-123.121817
2,Dunsmuir & Beatty,49.279764,-123.110154,26,49.279764%2C-123.110154
3,12th & Yukon (City Hall),49.260599,-123.113504,16,49.260599%2C-123.113504
4,8th & Ash,49.264215,-123.117772,16,49.264215%2C-123.117772
5,Spyglass & Seawall,49.270877,-123.115103,14,49.270877%2C-123.115103
6,Stamps Landing,49.268930,-123.118130,14,49.26893%2C-123.11813
7,Ontario & Seawall,49.271721,-123.104207,20,49.271721%2C-123.104207
8,Canada Place,49.287214,-123.112870,26,49.287214%2C-123.11287
9,Granville & Georgia,49.282409,-123.118541,20,49.282409%2C-123.118541


In [99]:
fs_api_key = os.environ['FOURSQUARE_KEY_2']
foursq_dict = {'Category':[], 'Name': [], 'Assc Bike Lat': [], 'Assc Bike Long': [] }
category_map = {0: 'Arts & Entertainment', 1: 'Schools', 2: 'Municipal', 3: 'Cafes', 4: 'Restaurants', 5: 'Bars',
                6: 'Landmarks, Parks & Sports', 7: 'Retail (Food/Bev)', 8: 'Drug and Convenience Stores', 9: 'Retail (Clothing)', 10: 'Retail (Other goods)'}


# target POI categories from FourSquare
arts_ent = '10000'
schools = '12014%2C12059%2C' 
municipal = '12004%2C12080%2C19046%2C15014'
cafes = '13032'
restaurants = '13065'
bars = '13029%2C13003'
outdoors = '16000%2C18000'
retail_food = '17057'
retail_drug_conv = '17029%2C17035'
retail_clothes = '17043'
retail_goods = '17003%2C17016%2C17018%2C17022%2C17023%2C17033%2C17114'

for location in bike_df['Coord_String'].values:
    # For current bike station location, loop through categories and send request for each category
    categories = [arts_ent, schools, municipal, cafes, restaurants, bars, outdoors, retail_food, retail_drug_conv, retail_clothes, retail_goods]
    responses = []
    for category in categories:
        url = "https://api.foursquare.com/v3/places/search?radius=300&limit=50&ll=" + location + '&categories=' + category

        headers = {"accept": "application/json",
                'Authorization': fs_api_key}

        response = requests.get(url, headers=headers)
        responses.append(response)
        print(location, category, response.status_code)

    # loop through responses to pull out results.
    for response in responses:
        for place in response.json()['results']:
            # pull name
            name = place['name']
            foursq_dict['Name'].append(name)

            # find coordinates of associated bike station
            station_lat = response.json()['context']['geo_bounds']['circle']['center']['latitude']
            station_long = response.json()['context']['geo_bounds']['circle']['center']['longitude']
            foursq_dict['Assc Bike Lat'].append(station_lat)
            foursq_dict['Assc Bike Long'].append(station_long)

            # find category
            category = category_map[responses.index(response)]
            foursq_dict['Category'].append(category)


49.262487%2C-123.114397 10000 200
49.262487%2C-123.114397 12014%2C12059%2C 200
49.262487%2C-123.114397 12004%2C12080%2C19046%2C15014 200
49.262487%2C-123.114397 13032 200
49.262487%2C-123.114397 13065 200
49.262487%2C-123.114397 13029%2C13003 200
49.262487%2C-123.114397 16000%2C18000 200
49.262487%2C-123.114397 17057 200
49.262487%2C-123.114397 17029%2C17035 200
49.262487%2C-123.114397 17043 200
49.262487%2C-123.114397 17003%2C17016%2C17018%2C17022%2C17023%2C17033%2C17114 200
49.274566%2C-123.121817 10000 200
49.274566%2C-123.121817 12014%2C12059%2C 200
49.274566%2C-123.121817 12004%2C12080%2C19046%2C15014 200
49.274566%2C-123.121817 13032 200
49.274566%2C-123.121817 13065 200
49.274566%2C-123.121817 13029%2C13003 200
49.274566%2C-123.121817 16000%2C18000 200
49.274566%2C-123.121817 17057 200
49.274566%2C-123.121817 17029%2C17035 200
49.274566%2C-123.121817 17043 200
49.274566%2C-123.121817 17003%2C17016%2C17018%2C17022%2C17023%2C17033%2C17114 200
49.279764%2C-123.110154 10000 200
49.2

In [100]:
foursq_df = pd.DataFrame(foursq_dict)

In [102]:
foursq_df.to_csv('foursquare_df', index=False)

In [86]:
testdf.dtypes

Category           object
Name               object
Assc Bike Lat     float64
Assc Bike Long    float64
dtype: object

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

active
arts
education
breweries
hospitals
metrostations
bars
communitycenters
landmarks
libraries
restaurants
shopping

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating